In [1]:
'''
Task 3: playing with NN framwork/keras and basic sentiment analysis
- use the following model as a baseline and improve it!
- export your metadata (just basic hyperparameters and outcomes for test data!)
- test data = 0.3 (not in this example, change it!)
- random_state = 4222
- no need to cross-validation!
'''

'\nTask 3: playing with NN framwork/keras and basic sentiment analysis\n- use the following model as a baseline and improve it!\n- export your metadata (just basic hyperparameters and outcomes for test data!)\n- test data = 0.3 (not in this example, change it!)\n- random_state = 4222\n- no need to cross-validation!\n'

In [2]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, GRU, Convolution1D, GlobalMaxPooling1D, Activation
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


**Setting parameters**

In [3]:
# parameters
max_fatures = 500
embed_dim = 128
lstm_out = 196
dropout = 0.1
dropout_1d = 0.4
recurrent_dropout = 0.1
random_state = 1324
validation_size = 1000
batch_size = 16
epochs=2
verbose= 2

**Preparing data**

In [4]:
df = pd.read_csv('dataset_sentiment.csv')
df = df[['text','sentiment']]


df = df[df.sentiment != "Neutral"]
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply(lambda x: x.replace('rt',' '))
df['text'] = df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
print(df[0:10])

Y = pd.get_dummies(df['sentiment']).values
print(Y[0:10])

tok = Tokenizer(num_words=max_fatures, split=' ')
tok.fit_on_texts(df['text'].values)
X = tok.texts_to_sequences(df['text'].values)
print(X[0:1])
X = pad_sequences(X)
print(X[0:1])

                                                 text sentiment
1     scottwalker didnt catch the full gopdebate l...  Positive
3     robgeorge that carly fiorina is trending  ho...  Positive
4     danscavino gopdebate w realdonaldtrump deliv...  Positive
5     gregabbott_tx tedcruz on my first day i will...  Positive
6     warriorwoman91 i liked her and was happy whe...  Negative
8   deer in the headlights   lizzwinstead ben cars...  Negative
9     nancyosborne180 last nights debate proved it...  Negative
10  jgreendc realdonaldtrump in all fairness billc...  Negative
11    waynedupreeshow just woke up to tweet this o...  Positive
12  me reading my familys comments about how great...  Negative
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]
[[363, 122, 1, 2, 39, 58, 237, 37, 210, 6, 174, 12]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 363 122   1   2
   39  58 237  37 210   6 174  12]]


## Creating the model (Please comment out necessary models while testing)

#### Model 1

In [5]:
nn = Sequential()
nn.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1], dropout=0.2))
nn.add(
    Convolution1D(
        nb_filter=128,
        filter_length=2,
        border_mode='valid',
        activation='relu',
        subsample_length=1,
    )
)
nn.add(GlobalMaxPooling1D())
nn.add(Dense(64))
nn.add(Dropout(0.2))
nn.add(Activation('relu'))
nn.add(Dense(2))
nn.add(Activation('softmax'))
nn.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(nn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 26, 128)           64000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 25, 128)           32896     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
__________

/Users/egebuyuksemerci/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/Users/egebuyuksemerci/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2, strides=1, padding="valid")`
  if __name__ == '__main__':


**Training the model 1**

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = random_state)
nn.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/2
 - 3s - loss: 0.4145 - acc: 0.8273
Epoch 2/2
 - 3s - loss: 0.3250 - acc: 0.8627


**Testing the model 1**

In [7]:
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score, accuracy = nn.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (accuracy))

score: 0.36
acc: 0.84


**Printing results for model 1**

In [8]:
pos_cnt, neg_cnt, pos_ok, neg_ok = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = nn.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0: neg_ok += 1
        else: pos_ok += 1
    if np.argmax(Y_validate[x]) == 0: neg_cnt += 1
    else: pos_cnt += 1

print("pos_acc", pos_ok/pos_cnt*100, "%")
print("neg_acc", neg_ok/neg_cnt*100, "%")

pos_acc 50.0 %
neg_acc 94.05940594059405 %


**Testing model 1 with a random sentence**

In [9]:
X2 = ['what are u going to say about that? the truth, wassock?!']
X2 = tok.texts_to_sequences(X2)
X2 = pad_sequences(X2, maxlen=26, dtype='int32', value=0)
print(X2)
print(nn.predict(X2, batch_size=1, verbose = 2)[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  48  37
  311 189   4 144  22  16   1 281]]
[ 0.96855766  0.0314423 ]


In [10]:
#### Model 2

In [11]:
nn = Sequential()
nn.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))

nn.add(GRU(units=32, name = "gru_1",return_sequences=True))
nn.add(GRU(units=16, name = "gru_2" ,return_sequences=True))
nn.add(GRU(units=8, name= "gru_3"))
nn.add(Dense(2, activation='sigmoid',name="dense_1"))
nn.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(nn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 26, 128)           64000     
_________________________________________________________________
gru_1 (GRU)                  (None, 26, 32)            15456     
_________________________________________________________________
gru_2 (GRU)                  (None, 26, 16)            2352      
_________________________________________________________________
gru_3 (GRU)                  (None, 8)                 600       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 18        
Total params: 82,426
Trainable params: 82,426
Non-trainable params: 0
_________________________________________________________________
None


**Training the model 2**

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = random_state)
nn.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/2
 - 74s - loss: 0.4350 - acc: 0.8274
Epoch 2/2
 - 81s - loss: 0.3650 - acc: 0.8538


**Testing the model 2**

In [13]:
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score, accuracy = nn.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (accuracy))

score: 0.38
acc: 0.84


**Printing results for model 2**

In [14]:
pos_cnt, neg_cnt, pos_ok, neg_ok = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = nn.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0: neg_ok += 1
        else: pos_ok += 1
    if np.argmax(Y_validate[x]) == 0: neg_cnt += 1
    else: pos_cnt += 1

print("pos_acc", pos_ok/pos_cnt*100, "%")
print("neg_acc", neg_ok/neg_cnt*100, "%")

pos_acc 45.83333333333333 %
neg_acc 94.80198019801979 %


**Testing model 2 with a random sentence**

In [15]:
X2 = ['what are u going to say about that? the truth, wassock?!']
X2 = tok.texts_to_sequences(X2)
X2 = pad_sequences(X2, maxlen=26, dtype='int32', value=0)
print(X2)
print(nn.predict(X2, batch_size=1, verbose = 2)[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  48  37
  311 189   4 144  22  16   1 281]]
[ 0.89606792  0.08673313]


In [16]:
#### Model 3

In [17]:
nn = Sequential()
nn.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
nn.add(SpatialDropout1D(dropout_1d))
nn.add(LSTM(lstm_out, dropout=dropout, recurrent_dropout=recurrent_dropout))
nn.add(Dense(2, activation='softmax'))
nn.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(nn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 26, 128)           64000     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 26, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 319,194
Trainable params: 319,194
Non-trainable params: 0
_________________________________________________________________
None


**Training the model 3**

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = random_state)
nn.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/2
 - 55s - loss: 0.4341 - acc: 0.8172
Epoch 2/2
 - 51s - loss: 0.3606 - acc: 0.8498


**Testing the model 3**

In [19]:
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score, accuracy = nn.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (accuracy))

score: 0.37
acc: 0.84


**Printing results for model 3**

In [20]:
pos_cnt, neg_cnt, pos_ok, neg_ok = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = nn.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0: neg_ok += 1
        else: pos_ok += 1
    if np.argmax(Y_validate[x]) == 0: neg_cnt += 1
    else: pos_cnt += 1

print("pos_acc", pos_ok/pos_cnt*100, "%")
print("neg_acc", neg_ok/neg_cnt*100, "%")

pos_acc 37.5 %
neg_acc 95.42079207920791 %


**Testing model 3 with a random sentence**

In [21]:
X2 = ['what are u going to say about that? the truth, wassock?!']
X2 = tok.texts_to_sequences(X2)
X2 = pad_sequences(X2, maxlen=26, dtype='int32', value=0)
print(X2)
print(nn.predict(X2, batch_size=1, verbose = 2)[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  48  37
  311 189   4 144  22  16   1 281]]
[ 0.88296747  0.11703251]
